In [1]:
from scraper import MangaScraper
from image_processory import OCR
from PIL import Image
import gradio as gr
import os

uploaded_images = []
languages_map = {
    "Hindi": [os.path.join('..', 'data', 'fonts', "NotoSansHindi-Regular.ttf"), "hi-IN"],
    "Kannada": [os.path.join('..', 'data', 'fonts', "NotoSansKannada-Regular.ttf"), "kn-IN"],
    "Bengali": [os.path.join('..', 'data', 'fonts', "NotoSansBengali-Regular.ttf"), "bn-IN"],
    "Gujarati": [os.path.join('..', 'data', 'fonts', "NotoSansGujarati-Regular.ttf"), "gu-IN"],
}
# Dummy function for translation (replace with actual translation logic)
def translate_manga():
    global uploaded_images
    ocr = OCR()
    translated_images = [ocr.translate_image(uploaded_image) for uploaded_image in uploaded_images]

    # for i in uploaded_images:
    #     translated_image = ocr.translate_image(i)
    #     translated_images.append(translated_image)

    return translated_images

def process_uploaded_files(url):
    global uploaded_images
    uploaded_images = []
    manga_scraper = MangaScraper()
    
    scraped_images_path = manga_scraper.scrape_manga_pill(url)
    if(scraped_images_path != False):
        print("Scraping Completed")
        images = [Image.open(file) for file in scraped_images_path]
        uploaded_images = scraped_images_path
    else:
        print("An error occurred while scraping the manga")
        images = []

    return images

# Gradio interface
with gr.Blocks() as manga_translation_ui:
    languages = ["Hindi", "Kannada", "Bengali", "Gujarati"]
    endpoints = ["Groq", "Sarvam"]
    gr.Markdown("## Manga Translation Program")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Original Manga Pages")
            original_images = gr.Gallery(label="Upload Manga Pages", interactive=False, height="900px")
        
        with gr.Column():
            gr.Markdown("### Translated Manga Pages")
            translated_images = gr.Gallery(label="Translated Manga Pages", interactive=False, height="900px")
    
    url_input = gr.Textbox(label="Enter Manga URL here...", placeholder="Enter Manga URL here...", lines=1)
    language_dropdown = gr.Dropdown(label="Select a language", choices=languages)
    endpoint_dropdown = gr.Dropdown(label="Select an endpoint", choices=endpoints)

    upload_btn = gr.Button("Upload Manga Pages")
    translate_btn = gr.Button("Translate")
   
    upload_btn.click(process_uploaded_files, inputs=[url_input], outputs=[original_images])
    translate_btn.click(fn=translate_manga, inputs=None, outputs=[translated_images])

# Launch the Gradio app
manga_translation_ui.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [2]:
from scraper import MangaScraper
from image_processory import OCR
from PIL import Image
import gradio as gr
import os

uploaded_images = []
languages_map = {
    "Hindi": [os.path.join('..', 'data', 'fonts', "NotoSansHindi-Regular.ttf"), "hi-IN"],
    "Kannada": [os.path.join('..', 'data', 'fonts', "NotoSansKannada-Regular.ttf"), "kn-IN"],
    "Bengali": [os.path.join('..', 'data', 'fonts', "NotoSansBengali-Regular.ttf"), "bn-IN"],
    "Gujarati": [os.path.join('..', 'data', 'fonts', "NotoSansGujarati-Regular.ttf"), "gu-IN"],
}

# Dummy function for translation (replace with actual translation logic)
def translate_manga():
    global uploaded_images
    ocr = OCR()
    
    # Iterate over images and yield the translated image one by one
    for uploaded_image in uploaded_images:
        translated_image = ocr.translate_image(uploaded_image)
        yield translated_image

def process_uploaded_files(url):
    global uploaded_images
    uploaded_images = []
    manga_scraper = MangaScraper()
    
    scraped_images_path = manga_scraper.scrape_manga_pill(url)
    if scraped_images_path != False:
        print("Scraping Completed")
        images = [Image.open(file) for file in scraped_images_path]
        uploaded_images = scraped_images_path
    else:
        print("An error occurred while scraping the manga")
        images = []

    return images

# Gradio interface
with gr.Blocks() as manga_translation_ui:
    languages = ["Hindi", "Kannada", "Bengali", "Gujarati"]
    endpoints = ["Groq", "Sarvam"]
    gr.Markdown("## Manga Translation Program")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Original Manga Pages")
            original_images = gr.Gallery(label="Upload Manga Pages", interactive=False, height="900px")
        
        with gr.Column():
            gr.Markdown("### Translated Manga Pages")
            translated_images = gr.Gallery(label="Translated Manga Pages", interactive=False, height="900px")
    
    url_input = gr.Textbox(label="Enter Manga URL here...", placeholder="Enter Manga URL here...", lines=1)
    language_dropdown = gr.Dropdown(label="Select a language", choices=languages)
    endpoint_dropdown = gr.Dropdown(label="Select an endpoint", choices=endpoints)

    upload_btn = gr.Button("Upload Manga Pages")
    translate_btn = gr.Button("Translate")
   
    upload_btn.click(process_uploaded_files, inputs=[url_input], outputs=[original_images])
    
    # The `translate_manga` now streams the results instead of returning them all at once
    translate_btn.click(fn=translate_manga, inputs=None, outputs=translated_images, stream=True)

# Launch the Gradio app
manga_translation_ui.launch(debug=True)

TypeError: EventListener._setup.<locals>.event_trigger() got an unexpected keyword argument 'stream'

In [1]:
from scraper import MangaScraper
from image_processory import OCR
from PIL import Image
import gradio as gr
import os

uploaded_images = []

# Dummy function for translation (replace with actual translation logic)
def translate_manga(language):
    print("Translating manga to", language)
    global uploaded_images
    ocr = OCR()
    
    translated_images = []
    # Iterate over images and yield the translated image one by one
    for uploaded_image in uploaded_images:
        translated_image = ocr.translate_image(uploaded_image, language=language)
        translated_images.append(translated_image)
        # Use gr.update to progressively update the gallery
        yield gr.update(value=translated_images)

def process_uploaded_files(url):
    global uploaded_images
    uploaded_images = []
    manga_scraper = MangaScraper()
    
    scraped_images_path = manga_scraper.scrape_manga_pill(url)
    if scraped_images_path != False:
        print("Scraping Completed")
        images = [Image.open(file) for file in scraped_images_path]
        uploaded_images = scraped_images_path
    else:
        print("An error occurred while scraping the manga")
        images = []

    return images

# Gradio interface
with gr.Blocks() as manga_translation_ui:
    languages = ["Hindi", "Kannada", "Bengali", "Gujarati"]
    endpoints = ["Groq", "Sarvam"]
    gr.Markdown("## Manga Translation Program")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Original Manga Pages")
            original_images = gr.Gallery(label="Upload Manga Pages", interactive=False, height="900px")
        
        with gr.Column():
            gr.Markdown("### Translated Manga Pages")
            translated_images = gr.Gallery(label="Translated Manga Pages", interactive=False, height="900px")
    
    url_input = gr.Textbox(label="Enter Manga URL here...", placeholder="Enter Manga URL here...", lines=1)
    language_dropdown = gr.Dropdown(label="Select a language", choices=languages, value=languages[0])
    endpoint_dropdown = gr.Dropdown(label="Select an endpoint", choices=endpoints, value=endpoints[0])

    upload_btn = gr.Button("Upload Manga Pages")
    translate_btn = gr.Button("Translate")
   
    upload_btn.click(process_uploaded_files, inputs=[url_input], outputs=[original_images])
    
    # No need for `stream=True`, use generator for progressively updating output

    translate_btn.click(fn=translate_manga, inputs=[language_dropdown], outputs=[translated_images])

# Launch the Gradio app
manga_translation_ui.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


<img alt="Chainsaw Man Chapter 180 Page 1" class="js-page" data-src="https://cdn.readdetectiveconan.com/file/mangap/723/10180000/1.jpeg?t=1729005075" height="1145" loading="lazy" width="784"/>
chainsaw-man-chapter-180
dir_path: ../data/images/chainsaw-man-chapter-180
Creation of the directory failed
[Errno 17] File exists: '../data/images/chainsaw-man-chapter-180'
img_path:  ../data/images/chainsaw-man-chapter-180/1.jpeg?t=1729005075
img_path:  ../data/images/chainsaw-man-chapter-180/2.jpeg?t=1729005075
img_path:  ../data/images/chainsaw-man-chapter-180/3.jpeg?t=1729005075
img_path:  ../data/images/chainsaw-man-chapter-180/4.jpeg?t=1729005075
img_path:  ../data/images/chainsaw-man-chapter-180/5.jpeg?t=1729005075
img_path:  ../data/images/chainsaw-man-chapter-180/6.jpeg?t=1729005075
img_path:  ../data/images/chainsaw-man-chapter-180/7.jpeg?t=1729005075
img_path:  ../data/images/chainsaw-man-chapter-180/8.jpeg?t=1729005075
img_path:  ../data/images/chainsaw-man-chapter-180/9.jpeg?t=17290